## Day 22 - 3D sand block simulation... :panik:

In [41]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['1,0,1~1,2,1',
 '0,0,2~2,0,2',
 '0,2,3~2,2,3',
 '0,0,4~0,2,4',
 '2,0,5~2,2,5',
 '0,1,6~2,1,6',
 '1,1,8~1,1,9']

Each row describes the coordinates of a block, starting from lowest z coord to highest.
- format is x1,y1,z1~x2,y2,z2 to describe the volume that block spans

Blocks are supported if there's at least one contact point below, otherwise they fall.

**Part 1: Simulate such that all the blocks settle (some haven't 'landed' down yet) and then find out how many blocks can be removed without things falling down!**

In [77]:
from dataclasses import dataclass

@dataclass
class Brick():
    x: tuple[int, int]
    y: tuple[int, int]
    z: tuple[int, int]

    def can_balance_on_this(self, other_brick: 'Brick'):
        z1s, z1e = self.z
        z2s, z2e = other_brick.z

        return z2s in range(z1s, z1e) or z2e in range(z1s, z1e)

    def fall_vertically_to_row(self, row: int):
        z1, z2 = self.z
        assert row < z1, f"{row=} {z1=}"
        diff = z1 - row
        self.z = (z1-diff, z2-diff)
    
    def overlap_horizontally(self, other_brick: 'Brick'):
        return (
            (
            other_brick.x[0] in range(self.x[0], self.x[1]+1) or
            other_brick.x[1] in range(self.x[0], self.x[1]+1) or
            self.x[0] in range(other_brick.x[0], other_brick.x[1]+1) or
            self.x[1] in range(other_brick.x[0], other_brick.x[1]+1)
            ) and
            (
            other_brick.y[0] in range(self.y[0], self.y[1]+1) or
            other_brick.y[1] in range(self.y[0], self.y[1]+1) or
            self.y[0] in range(other_brick.y[0], other_brick.y[1]+1) or
            self.y[1] in range(other_brick.y[0], other_brick.y[1]+1)
            )
        )

In [108]:
class Stack():

    def __init__(self, bricks: list[Brick]):
        self.bricks: list[Brick] = bricks
        bricks.sort(key=lambda b: b.z[0])
        self.shift_bricks_down_as_far_as_they_can()

    
    def shift_bricks_down_as_far_as_they_can(self):
        z_max = max([brick.z[1] for brick in self.bricks])

        def set_flat_rows() -> dict[int, list[Brick]]:
            flat_rows = {z:[] for z in range(1, z_max+1)}
            for brick in self.bricks:
                for z in range(brick.z[0], brick.z[1]+1):
                    flat_rows[z].append(brick)
            return flat_rows
        
        flat_rows = set_flat_rows()

        for brick in self.bricks:
            for z in range(brick.z[0]-1, 0, -1):
                doesnt_overlap = True
                for sub_brick in flat_rows[z]:
                    if brick != sub_brick:
                        doesnt_overlap = not sub_brick.overlap_horizontally(brick)
                if doesnt_overlap:
                    brick.fall_vertically_to_row(z)
                    flat_rows = set_flat_rows()
                else:
                    # must be an overlap
                    break


In [109]:
example_bricks = []
for line in example_lines:
    l, r = line.split("~")
    lx, ly, lz = map(int, l.split(","))
    rx, ry, rz = map(int, r.split(","))
    example_bricks.append(
        Brick((lx, rx), (ly, ry), (lz, rz))
    )
example_stack = Stack(example_bricks)

In [111]:
# SET UP (slow to run so get it defined etc.)

assert False, "Put it here to avoid overwriting input_stack lol. Can't be waiting another couple minutes!"

bricks = []
for line in input_lines:
    l, r = line.split("~")
    lx, ly, lz = map(int, l.split(","))
    rx, ry, rz = map(int, r.split(","))
    bricks.append(
        Brick((lx, rx), (ly, ry), (lz, rz))
    )
input_stack = Stack(bricks)

In [125]:
def part1(example: bool = True):
    if example:
        stack = example_stack
    else:
        stack = input_stack

    
    z_max = max([brick.z[1] for brick in stack.bricks])
    flat_rows = {z:[] for z in range(1, z_max+1)}
    for brick in stack.bricks:
        for z in range(brick.z[0], brick.z[1]+1):
            flat_rows[z].append(brick)

    total = 0
    for brick in stack.bricks:
        z_above = brick.z[1] + 1
        if z_above in flat_rows:
            stability = []
            for other_brick in flat_rows[z_above]:
                z_below_other = other_brick.z[0] - 1
                if z_below_other == 0:
                    stability.append(True)
                    break
                assert isinstance(other_brick, Brick)
                stability.append(
                    any(
                        other_brick.overlap_horizontally(support_brick)
                        for support_brick in flat_rows[z_below_other]
                        if brick != support_brick and support_brick.z[1] == z_below_other
                    )
                )
            if all(stability):
                total += 1
        else:
            # top rows can always be removed right?
            total += 1
    
    return total               
                
    

assert part1(example=True)
part1(example = False) # on real input, 974 is TOO HIGH


973